In [5]:
import datetime as dt
import time
import random
import logging
from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick
from IPython.display import clear_output
import threading
import warnings
from optibook.common_types import SocialMediaFeed
import torch
from transformers import BertModel, BertTokenizerFast
import torch.nn as nn
import pickle
import numpy as np

logging.getLogger('client').setLevel('ERROR')   

# Seperated from main functionality so connection has to be established once to avoid problems with Cloud 9
exchange = Exchange()
exchange.connect()

MIN_SELLING_PRICE = 0.10
MAX_BUYING_PRICE = 100000.00
QUOTED_VOLUME = 10
FIXED_MINIMUM_CREDIT = 0.15
PRICE_RETREAT_PER_LOT = 0.005
POSITION_LIMIT = 100
THRESHOLD = 0.1

2023-11-19 08:30:15,540 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [6]:
print('start')

def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

        INSTRUMENTS = exchange.get_instruments()
        
def closing():
    positions = exchange.get_positions()
    pnl = exchange.get_pnl()

    #print(f'Positions before: {positions}')
    #print(f'\nPnL before: {pnl:.2f}')

    #print(f'\nTrading out of positions')
    for iid, pos in positions.items():
        if pos > 0:
            #print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
            exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
        elif pos < 0:
            #print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
            exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
        else:
            pass
            #print(f'-- No initial position in {iid}, skipping..')
        time.sleep(0.10)

    time.sleep(1.0)
    positions = exchange.get_positions()
    pnl = exchange.get_pnl()
    #print(f'\nPositions after: {positions}')
    #print(f'\nPnL after: {pnl:.2f}')

closing()
print(exchange.get_positions())

start
{'NVDA': 0, 'ING': 0, 'SAN': 0, 'PFE': 0, 'NVDA_B': 0, 'CSCO': 0, 'ING_B': 0, 'PFE_B': 0, 'SAN_B': 0, 'CSCO_B': 0}


In [7]:
np.random.seed(42)
warnings.filterwarnings('ignore')

#------------------------Setup--------------------------#
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

class CustomBERTModel(nn.Module):
    def __init__(self):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained('prajjwal1/bert-tiny')
        self.classifier_stock = nn.Linear(self.bert.config.hidden_size, 6) # stock classification head
        self.classifier_sentiment = nn.Linear(self.bert.config.hidden_size, 3) # sentiment classification head
        self.regression = nn.Linear(self.bert.config.hidden_size, 1) # regression head
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_output = outputs[1]
        stock_labels = self.classifier_stock(pooled_output)
        sentiment_labels = self.classifier_sentiment(pooled_output)
        regression_values = self.regression(pooled_output)
        
        return stock_labels, sentiment_labels, regression_values

deployed_model = CustomBERTModel()
deployed_model.load_state_dict(torch.load("082_081.pt"))
deployed_model = deployed_model.to("cpu")

2023-11-19 08:30:38,068 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 08:30:38,193 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
2023-11-19 08:30:38,201 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 08:30:38,586 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expe

RuntimeError: Error(s) in loading state_dict for CustomBERTModel:
	Missing key(s) in state_dict: "bert.embeddings.position_ids". 

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-tiny')
def prepare_input(text, tokenizer):
    """
    Takes a string, tokenizes, and prepares it into expected format (list of token ids, attention masks, etc.) ready for model input

    Arguments:
    text -- string, Raw text string
    tokenizer -- transformers.Tokenizer, Initialized tokenizer

    Returns:
    input_dict -- dictionary, Contains required inputs for model
    """
    # Tokenize the text
    encoding = tokenizer.encode_plus(
        text,
        truncation=True, 
        padding=True,
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Get the input ids and attention masks from tokenizer and convert to tensors
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Put all tensor entries into a single dictionary
    input_dict = {
        'input_ids': input_ids,
        'token_type_ids': torch.zeros(input_ids.shape, dtype=torch.long),
        'attention_mask': attention_mask,
    }
    
    return input_dict

def predict_loaded_from_loaded_model(model, res, scaler):
    model.eval()
    with torch.no_grad():
        inputs = {key: val.to("cpu") for key, val in res.items()}  

        stock_labels_pred, sentiment_labels_pred, regression_values_pred = model(inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"])
        
        stock_label = torch.argmax(stock_labels_pred, dim=1).item()
        sentiment_label = torch.argmax(sentiment_labels_pred, dim=1).item()
        regression_value = scaler.inverse_transform(regression_values_pred.cpu().numpy()) # inverse transform of scaling

    return stock_label, sentiment_label, regression_value[0][0]  # return the single value 


2023-11-19 08:30:42,673 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 08:30:42,795 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
2023-11-19 08:30:42,804 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 08:30:42,924 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/vocab.txt HTTP/1.1" 200 0
2023-11-19 08:30:42,930 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 08:30:43,049 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/tokenizer.json HTTP/1.1" 404 0
2023-11-19 08:30:43,054 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 08:30:43,172 [urllib3.connect

In [9]:
old_timestamp = None

def news():
    social_feeds = exchange.poll_new_social_media_feeds()
    
    if not social_feeds:
        print(f'{dt.datetime.now()}: no new messages')
    else:
        for feed in social_feeds:
            print(f'{feed.timestamp}: {feed.post}')
            input = prepare_input(feed.post, tokenizer)
            tock_label, sentiment_label, regression_value = predict_loaded_from_loaded_model(deployed_model, input ,scaler)
            print(tock_label, sentiment_label, regression_value)
            if sentiment_label == 0:
                pass
            else:
                map = ['None', 'NVDA', 'ING', 'SAN', 'PFE', 'CSCO']
                map_1 = ['None', 'NVDA_B', 'ING_B', 'SAN_B', 'PFE_B', 'CSCO_B']
                stock = map[tock_label]
                stock_b = map_1[tock_label]
                print(stock)
                if stock == 'None':
                    pass
                else:
                    stock_book = exchange.get_last_price_book(stock)
                    stock_book_b = exchange.get_last_price_book(stock_b)
                    if regression_value > 0:
                        # We first try to calculate the target price based on the models prediction
                        exchange.insert_order(stock, price=MAX_BUYING_PRICE, volume=99, side='bid', order_type='ioc')
                        exchange.insert_order(stock_b, price=MAX_BUYING_PRICE, volume=99, side='bid', order_type='ioc')
                        time.sleep(8)
                        if stock_book.bids and stock_book.asks and stock_book_b.bids and stock_book_b.asks: 
                            best_bid_price_A = stock_book.bids[0].price    
                            best_ask_price_A = stock_book.asks[0].price
                            mid_price_A = (best_bid_price_A + best_ask_price_A) / 2.0
                            target_price = mid_price_A + mid_price_A * float(np.exp(regression_value))
                            position_A = abs(exchange.get_positions()[stock])
                            #volume_A = -position_A
                            exchange.insert_order(stock, price=round(target_price, 2), volume=position_A, side='ask', order_type='limit')
                            exchange.insert_order(stock_b, price=round(target_price, 2), volume=position_A, side='ask', order_type='limit')
                            time.sleep(10)
                            exchange.delete_orders(stock)
                            exchange.delete_orders(stock_b)
                            closing()
                        else:
                            time.sleep(10)
                            exchange.delete_orders(stock)
                            exchange.delete_orders(stock_b)
                            closing()
                            # set limit
                            # after 10 sec not filled force close
                    elif regression_value < 0:
                        exchange.insert_order(stock, price=MIN_SELLING_PRICE, volume=99, side='ask', order_type='ioc')
                        exchange.insert_order(stock_b, price=MIN_SELLING_PRICE, volume=99, side='ask', order_type='ioc')
                        time.sleep(8)
                        if stock_book.bids and stock_book.asks and stock_book_b.bids and stock_book_b.asks:  
                            best_bid_price_A = stock_book.bids[0].price    
                            best_ask_price_A = stock_book.asks[0].price
                            mid_price_A = (best_bid_price_A + best_ask_price_A) / 2.0
                            target_price = mid_price_A + mid_price_A * float(np.exp(regression_value))
                            position_A = abs(exchange.get_positions()[stock])
                            exchange.insert_order(stock, price=round(target_price, 2), volume=position_A , side='bid', order_type='limit')
                            exchange.insert_order(stock_b, price=round(target_price, 2), volume=position_A , side='bid', order_type='limit')
                            time.sleep(10)
                            exchange.delete_orders(stock)
                            exchange.delete_orders(stock_b)
                            closing()
                        else:
                            time.sleep(10)
                            exchange.delete_orders(stock)
                            exchange.delete_orders(stock_b)
                            closing()
    #time.sleep(0.5)

In [ ]:
while True:
    news()
    time.sleep(1)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


2023-11-19 08:30:29.621117: @TechGossips: Unverified reports suggesting Nvidia might reduce customer support hours circulating. #CustomerSupport #Rumors
0 0 0.005768277
2023-11-19 08:30:41.595663: @TheEconomist: Despite all the noise, today's interest rate announcement will have little impact on stocks. #InterestRates #StockMarket
0 0 -0.0011032061
2023-11-19 08:30:53.254139: no new messages
2023-11-19 08:30:54.255503: no new messages
2023-11-19 08:30:55.256118: no new messages
2023-11-19 08:30:56.257293: no new messages
2023-11-19 08:30:57.258492: no new messages
2023-11-19 08:30:58.259810: no new messages
2023-11-19 08:30:59.261184: no new messages
2023-11-19 08:31:00.262389: no new messages
2023-11-19 08:31:01.264063: no new messages
2023-11-19 08:31:02.265291: no new messages
2023-11-19 08:31:03.266492: no new messages
2023-11-19 08:31:04.267728: no new messages
2023-11-19 08:31:05.269559: no new messages
2023-11-19 08:31:06.270789: no new messages
2023-11-19 08:31:07.272071: no ne

# Order Execution

# Helpful printouts to understand the data structure

# Close all positions for a the current ask/bid price

In [ ]:
def arbitrage(stock_a, stock_b):
    while True:
        stock_A_book = exchange.get_last_price_book(stock_a)
        stock_B_book = exchange.get_last_price_book(stock_b)
        if stock_A_book.bids and stock_A_book.asks and stock_B_book.bids and stock_B_book.asks:
            best_bid_price_A = stock_A_book.bids[0].price    
            best_ask_price_A = stock_A_book.asks[0].price
            mid_price_A = (best_bid_price_A + best_ask_price_A) / 2.0
            best_bid_price_B = stock_B_book.bids[0].price
            best_ask_price_B = stock_B_book.asks[0].price
            mid_price_B = (best_bid_price_B + best_ask_price_B) / 2.0 
            diff = mid_price_A - mid_price_B
            
            # Obtain own current position in instrument
            position_A = exchange.get_positions()[stock_a]
            position_B = exchange.get_positions()[stock_b]
            # Calculate our fair/theoretical price based on the market mid price and our current position
            theoretical_price_A = mid_price_A - PRICE_RETREAT_PER_LOT * position_A
            theoretical_price_B = mid_price_B - PRICE_RETREAT_PER_LOT * position_B

            # Calculate final bid and ask prices to insert
            bid_price_A = round_down_to_tick(theoretical_price_A - FIXED_MINIMUM_CREDIT, exchange.get_instruments()[stock_a].tick_size)
            bid_price_B = round_down_to_tick(theoretical_price_B - FIXED_MINIMUM_CREDIT, exchange.get_instruments()[stock_b].tick_size)
            
            ask_price_A = round_up_to_tick(theoretical_price_A + FIXED_MINIMUM_CREDIT, exchange.get_instruments()[stock_a].tick_size)
            ask_price_B = round_up_to_tick(theoretical_price_B + FIXED_MINIMUM_CREDIT, exchange.get_instruments()[stock_b].tick_size)

            # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
            max_volume_to_buy_A = POSITION_LIMIT - position_A
            max_volume_to_buy_B = POSITION_LIMIT - position_B
            
            max_volume_to_sell_A = POSITION_LIMIT + position_A
            max_volume_to_sell_B = POSITION_LIMIT + position_B

            bid_volume_A = min(QUOTED_VOLUME, max_volume_to_buy_A)
            bid_volume_B = min(QUOTED_VOLUME, max_volume_to_buy_B)
            
            ask_volume_A = min(QUOTED_VOLUME, max_volume_to_sell_A)
            ask_volume_B = min(QUOTED_VOLUME, max_volume_to_sell_B)
            
            #print(diff)
            if abs(diff) >= 0.2:
                print('Success')
                if diff > 0:
                    # CSCO
                    #exchange.insert_order(exchange.get_instruments()['CSCO'], price=MAX_BUYING_PRICE, volume=10, side='bid', order_type='ioc')
                    #exchange.insert_order(exchange.get_instruments()['CSCO_B'], price=MAX_BUYING_PRICE, volume=10, side='ask', order_type='ioc')
                    insert_quotes(exchange, exchange.get_instruments()[stock_a], best_bid_price_A, best_ask_price_A, bid_volume_A, 0)
                    insert_quotes(exchange, exchange.get_instruments()[stock_b], best_bid_price_B, best_ask_price_B, 0, ask_volume_B)
                    print(stock_a, best_bid_price_A, best_ask_price_A, bid_volume_A, 0)
                    print(stock_b, best_bid_price_B, best_ask_price_B, 0, ask_volume_B)
                    time.sleep(60)
                    exchange.delete_orders(exchange.get_instruments()[stock_b].instrument_id)
                    exchange.delete_orders(exchange.get_instruments()[stock_a].instrument_id)
                    closing()
                else:
                    # CSCO_B
                    insert_quotes(exchange, exchange.get_instruments()[stock_b], best_bid_price_B, best_ask_price_B, bid_volume_B, 0)
                    insert_quotes(exchange, exchange.get_instruments()[stock_a], best_bid_price_A, best_ask_price_A, 0, ask_volume_A)
                    print(stock_b, best_bid_price_B, best_ask_price_B, bid_volume_B, 0)
                    print(stock_a, best_bid_price_A, best_ask_price_A, 0, ask_volume_A)
                    time.sleep(60)
                    exchange.delete_orders(exchange.get_instruments()[stock_b].instrument_id)
                    exchange.delete_orders(exchange.get_instruments()[stock_a].instrument_id)
                    closing()
            else:
                #print('no success')
                closing()
                #print(exchange.get_positions().values())
                # pass
        time.sleep(0.5)

In [ ]:
def start_multithreading():
    thread1 = threading.Thread(target=arbitrage, args=("CSCO", "CSCO_B"))
    thread2 = threading.Thread(target=arbitrage, args=('ING', 'ING_B'))
    thread3 = threading.Thread(target=arbitrage, args=('NVDA','NVDA_B'))
    thread4 = threading.Thread(target=arbitrage, args=('PFE', 'PFE_B'))
    thread5 = threading.Thread(target=arbitrage, args=('SAN', 'SAN_B'))
    
    thread1.start()
    thread2.start()
    thread3.start()
    thread4.start()
    thread5.start()
    
    thread2.join()
    thread1.join()
    thread3.join()
    thread4.join()
    thread5.join()

In [ ]:
start_multithreading()